# CALBC vs 8.1 Sketch Model Test

This code tests the CALBC version 8.1 Sketch Model. It utilizes a template file, which is designed to gather test data, and applies this data to the sketch model to generate final results. Test template can be updated [here](https://caltrans.sharepoint.com/:x:/r/sites/DOTHQPMPCalBCUpdate/Shared%20Documents/General/Testbed/Input/testbed_template.xlsm?d=w3bca22e7b5714f5399bb4015c94aa134&csf=1&web=1&e=9AuFn9). To run this code, you'll need to sync the [Testbed folder](https://teams.microsoft.com/l/team/19%3AQrQ3LladNmkiVpFmuCmW2H72CKQz9MrBc2yr1kukBoA1%40thread.tacv2/conversations?groupId=f2d54ae5-490b-4b95-99cf-bafddf8f38b6&tenantId=621b0a64-1740-43cc-8d88-4540d3487556) from [CalB/C Update](https://teams.microsoft.com/l/team/19%3AQrQ3LladNmkiVpFmuCmW2H72CKQz9MrBc2yr1kukBoA1%40thread.tacv2/conversations?groupId=f2d54ae5-490b-4b95-99cf-bafddf8f38b6&tenantId=621b0a64-1740-43cc-8d88-4540d3487556) Teams folder to your local machine. Output will be saved in designated [Output Folder](https://caltrans.sharepoint.com/:f:/r/sites/DOTHQPMPCalBCUpdate/Shared%20Documents/General/Testbed/Output?csf=1&web=1&e=X5VbAM). 

Steps: 
- Update relevant tables (e.g., cost) in the main file using the template.
- Save a separate file for each project (e.g., 1.0, 2.0, etc.) in the designated output directory.
- Update input values in the output files using the relevant named ranges.
- Recalculate formulas and save the final version of each updated project file. 

In [ ]:
pip install xlwings 

In [ ]:
pip install openpyxl 

In [3]:
import openpyxl
import xlwings as xw
import pandas as pd
import shutil
import io
import os
import pandas as pd
import time


In [4]:
user_profile = os.getenv('USERPROFILE') ## Retrieving user profile directory from 'USERPROFILE' environment variable.

In [5]:
# Constructing the base directory path using the user profile directory.
base_dir = os.path.join(
    user_profile, 
    'California Department of Transportation',
    'DOT HQ PMP Cal B C Update - General',
    'Testbed'
)

In [6]:
# Constructing file directory for output and reading input files.
output_dir = os.path.join(base_dir, 'Output')
sketch_file = os.path.join(base_dir, 'Input', 'cal-bc-8-1-sketch-a11y.xlsm')
template_file = os.path.join(base_dir, 'Input', 'testbed_template.xlsm')

## Update Cost Table

In [7]:
def get_tables_by_prefix(template_wb, prefix):
    # Initialize an empty list to store the tables that match the prefix
    tables_to_copy = []
    
    for sheet in template_wb.sheets:
        # Iterate through each table (ListObject) in the current sheet
        for table in sheet.tables:
            if table.name.startswith(prefix):
                tables_to_copy.append((sheet.name, table.name))
                
    # Return the list of tables that matched the prefix
    return tables_to_copy

In [8]:
# Test function for get_tables_by_prefix
def test_get_tables_by_prefix():

    with xw.App(visible=False) as app:
        template_wb = app.books.open(template_file)
        
        table_prefix = '_ConsCost'
        

        tables_to_copy = get_tables_by_prefix(template_wb, table_prefix)
        

        if tables_to_copy:
            print(f"Tables found with prefix '{table_prefix}':")
            for sheet_name, table_name in tables_to_copy:
                print(f"Sheet: {sheet_name}, Table: {table_name}")
        else:
            print(f"No tables found with prefix '{table_prefix}'.")

        template_wb.close()

In [ ]:
test_get_tables_by_prefix()

In [10]:
def update_cost_in_main_file(template_file, sketch_file, output_dir, tables_info):   
    # Open the template file and the main file
    with xw.App(visible=False) as app:
        template_wb = app.books.open(template_file)
        main_wb = app.books.open(sketch_file)

        # Iterating over all table types
        for table_type, target_range in tables_info.items():
            # Iterating through suffixes
            suffixes = ['1.0', '2.0', '3.0'] 
            
            for suffix in suffixes:
                print(f"Processing table type '{table_type}' with suffix '{suffix}'...")
                
                # Getting the tables with the specified prefix and suffix
                tables_to_copy = get_tables_by_prefix(template_wb, f"{table_type}{suffix}")
                
                if not tables_to_copy:
                    print(f"No tables found with prefix '{table_type}{suffix}'.")
                    continue
                
                for sheet_name, table_name in tables_to_copy:
                    sheet = main_wb.sheets['1) Project Information']
                    target_range_obj = sheet.range(target_range) 
                    
                    # Getting the table from the template
                    table = template_wb.sheets[sheet_name].tables[table_name]
                    
                    # Getting the data from the table
                    table_data = table.range.value
                    
                    # Updating the target range in the main file
                    target_range_obj.value = table_data
                    
                    # Recalculating the workbook
                    main_wb.app.calculate()
                    
                    # Saving the updated file with the suffix
                    updated_file_path = os.path.join(output_dir, f"updated_file_{table_type}_{suffix}.xlsm")
                    
                    try:
                        # Attempting to save the workbook
                        main_wb.save(updated_file_path)
                        print(f"File saved as: {updated_file_path}")
                        
                        # Adding a short delay to ensure Excel has time to complete the save operation
                        time.sleep(4)
                        
                    except Exception as e:
                        print(f"Error saving file: {e}")
        
        # Closing the workbooks
        template_wb.close()
        main_wb.close()

In [11]:
tables_info = {
        '_ConsCost': 'V15:AC43',  # Same range for all ConsCost tables (1.0, 2.0, etc.)
    }

In [ ]:
update_cost_in_main_file(template_file, sketch_file, output_dir, tables_info)

## Updating Named Ranges

In [13]:
# Reading the Excel file as binary and loading it into a workbook object.
with open(sketch_file, 'rb') as f:
    file_data = f.read()
    
# Loading the workbook from the binary data, preserving any VBA code
    wb = openpyxl.load_workbook(io.BytesIO(file_data), keep_vba=True)

In [14]:
# function to find named ranges, extract sheet name and cell reference and update cell's value with corresponding value from name_value_map
def update_named_ranges(wb, name_value_map):
    for name, new_value in name_value_map.items():
        try:
            # Accessing the defined name ranges in the workbook via wb.defined_names
            defined_name = wb.defined_names[name]
            
            # Converting the generator to a list and get the first destination
            destinations = list(defined_name.destinations)
            if destinations:
                sheet_name, cell_reference = destinations[0]
                sheet = wb[sheet_name]
                
                # Updating the value of the cell
                sheet[cell_reference] = new_value
                
                # Verifying the updated value
                updated_value = sheet[cell_reference]
                print(f"Updated value of '{name}' in {sheet_name} ({cell_reference}): {updated_value}")
            else:
                print(f"Error: No destinations found for the named range '{name}'.")
        
        except KeyError:
            print(f"Error: Named range '{name}' not found in the workbook.")
        except Exception as e:
            print(f"An error occurred: {e}")

In [15]:
# Extracting project data from the template file
def extract_project_data_pandas(template_file):

    df = pd.read_excel(template_file, sheet_name="Overall Info")  
    df_filtered = df[df['ProjID'].notna()]  
    
    name_value_map = df_filtered.set_index('ProjID').to_dict(orient='index')
    
    return name_value_map


In [ ]:
# Using the extracting project function 
name_value_map = extract_project_data_pandas(template_file)

In [ ]:
#Checking generated dictionary 
print(name_value_map)

In [18]:
def recalculate_and_save_xlsm(name_value_map, output_dir):
    for proj_id, value in name_value_map.items():
        try:
            output_file_path = os.path.join(output_dir, f"updated_file__ConsCost_{proj_id}.xlsm")
            
            if os.path.exists(output_file_path):
                # Opening the workbook using openpyxl
                print(f"Opening workbook {output_file_path}...")
                
                wb_excel = openpyxl.load_workbook(output_file_path, keep_vba=True)  # keep_vba=True to preserve macros
                print(f"Workbook {output_file_path} opened successfully.")
                
                # Updating the named ranges in the workbook
                update_named_ranges(wb_excel, value)
                
                # Saving the workbook with the updated data
                wb_excel.save(output_file_path)
                print(f"Workbook updated and saved: {output_file_path}")
            else:
                print(f"Error: The file {output_file_path} does not exist.")
        
        except Exception as e:
            print(f"Error during recalculation for {proj_id}: {e}")

In [ ]:
recalculate_and_save_xlsm(name_value_map, output_dir)

In [ ]:
#Generate BCR Summary
def generate_BCR_summary(output_dir, template_file):
    try:
        #Extracting project IDs from the template file
        df = pd.read_excel(template_file, sheet_name="Overall Info")
        proj_ids = df['ProjID'].dropna().tolist()

        data = [] 

        #Looping through each project ID and process its corresponding file
        for proj_id in proj_ids:
            file_path = os.path.join(output_dir, f"updated_file__ConsCost_{proj_id}.xlsm")
            
            if os.path.exists(file_path):
                with xw.App(visible=False) as app:
                    wb = app.books.open(file_path)
                    
                    #List of named ranges we want to extract
                    named_ranges = ['BeneCostRatio', 'ReturnOnInvest', 'Payback']
                    
                    project_data = {'ProjID': proj_id}

                    # Looping through each named range and extract its value
                    for named_range in named_ranges:
                        if named_range in wb.names:
                            range_obj = wb.names[named_range]
                            range_value = range_obj.refers_to_range.value
                            project_data[named_range] = range_value
                        else:
                            project_data[named_range] = np.nan
                    
                    #Appending the collected data for the current project
                    data.append(project_data)
                    wb.close()

        if data:
            df_summary = pd.DataFrame(data)
            return df_summary
        else:
            return None
    
    except Exception:
        return None

In [ ]:
summary_df = generate_BCR_summary(output_dir, template_file)

In [ ]:
summary_df